In [1]:
"""
This is a python notebook to create a stock screener. The finished product will accept a set of parameters 
and output the stocks that meet those requirements. Currently this scanner follows the "Swing Traders Checklist" @
https://www.swing-trade-stocks.com/swing-traders-checklist.html. The goal is to deploy this with a suite of Azure Functions 
and use it in a suite of other related financial-python projects, such as twitter sentiment and FinViz sentiment analysis, 
using ML-clustering to define levels of support and resistance, and other ideas I might think of at 2am on a Sunday.
"""

'\nThis is a python notebook to create a stock screener. The finished product will accept a set of parameters \nand output the stocks that meet those requirements. Currently this scanner follows the "Swing Traders Checklist" @\nhttps://www.swing-trade-stocks.com/swing-traders-checklist.html. The goal is to deploy this to an Azure Function and \nuse it in a suite of other related financial-python projects, such as twitter sentiment and FinViz sentiment analysis, \nusing ML-clustering to define levels of support and resistance, and other ideas I might think of at 2am on a Sunday.\n'

In [1]:
import yahoo_fin.stock_info as si
import pandas as pd
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import talib
import requests
import json
import openpyxl
import psycopg2 as pg
import StockSentiment_FinViz.ipynb


ModuleNotFoundError: No module named 'StockSentiment_FinViz'

In [3]:

#Get connected 
conn = pg.connect("dbname=StonksGoUp user=postgres host=localhost password=admin")
cur = conn.cursor()

#read in the finnhub.io token
# This will be used until the ML aspect is tested and complete
with open('local_settings.txt') as f:
    json_local = json.load(f)

finn_token = json_local["finn_token"]

#define scanner parameters:
low = float(2.5)
high = float(25.0)
to = int(datetime.strptime(datetime.today().strftime("%d/%m/%Y") + " +0000", "%d/%m/%Y %z").timestamp())
fro = int((datetime.strptime(datetime.today().strftime("%d/%m/%Y") + " +0000", "%d/%m/%Y %z")-relativedelta(days=300)).timestamp())
earnings_period = int((datetime.strptime(datetime.today().strftime("%d/%m/%Y") + " +0000", "%d/%m/%Y %z")+relativedelta(days=5)).timestamp())
capital = 100000
risk = 0.05

get_tickers = """
    SELECT ticker from stockdata
    WHERE ticker IN (SELECT ticker from tickers)
    GROUP BY ticker
                 """
cur.execute(get_tickers, conn)

scanner_list = list([i[0] for i in cur.fetchall()])
print(scanner_list[:20])

# get sentiment table
get_sentiment = """
    SELECT ticker, round(avg(score), 2) as avg_sentiment 
    FROM sentiment 
    WHERE timestamp > current_date - interval '1 week' 
    GROUP BY ticker
    """
df_sentiment = pd.read_sql(get_sentiment, conn)
print(df_sentiment)

['A', 'AAL', 'AAP', 'ABBV', 'ABC', 'ABMD', 'ABT', 'ACN', 'ADBE', 'ADI', 'ADM', 'ADP', 'ADSK', 'AEE', 'AEP', 'AES', 'AFL', 'AIG', 'AIV', 'AIZ']
     sentimentid ticker                 timestamp  \
0              1   AMZN 2020-10-10 19:00:00+00:00   
1              2   AMZN 2020-10-10 16:06:00+00:00   
2              3   AMZN 2020-10-10 15:30:00+00:00   
3              4   AMZN 2020-10-10 14:08:00+00:00   
4              5   AMZN 2020-10-10 14:02:00+00:00   
..           ...    ...                       ...   
395          396     SQ 2020-09-03 16:03:00+00:00   
396          397     SQ 2020-09-03 15:31:00+00:00   
397          398     SQ 2020-09-03 10:08:00+00:00   
398          399     SQ 2020-09-03 10:00:00+00:00   
399          400     SQ 2020-09-02 15:39:00+00:00   

                                              headline   score  
0           Stocks That Would Have Made You Rich Today  0.5574  
1                   3 Top Tech Stocks to Buy Right Now  0.2023  
2    Is Arlo Leaving the 

In [14]:
# Functions for Technical Analysis

def get_hist(ticker, conn):
    # Get data from web-scraping
#     req = f"https://query1.finance.yahoo.com/v7/finance/download/{ticker}?period1={fro}&period2={to}&interval=1d&events=history"
#     data = pd.read_csv(req)
    
#     data.index = data["Date"].apply(lambda x: pd.Timestamp(x))
#     data.drop("Date", axis=1, inplace=True)
    
    # Get data from database
    get_data = f"""SELECT 
                    ticker
                    ,quotedate as "Date"
                    ,open as "Open"
                    ,high as "High"
                    ,low as "Low"
                    ,close as "Close"
                    ,adjclose as "Adj Close"
                    ,volume as "Volume"
                    FROM stockdata
                    WHERE ticker = '{ticker}'
                    ORDER BY quotedate ASC"""
    data = pd.read_sql(get_data, conn)
    
    return data

def get_indicators(data):
    # Get MACD
    data["macd"], data["macd_signal"], data["macd_hist"] = talib.MACD(data['Close'])
    
    # Get SMA10 and SMA30
    data["sma10"] = talib.SMA(data["Close"], timeperiod=10)
    data["sma30"] = talib.SMA(data["Close"], timeperiod=30)
    
    # Get MA200
    data["sma200"] = talib.SMA(data["Close"], timeperiod=200)
    
    # Get RSI
    data["rsi"] = talib.RSI(data["Close"])
    
    return data

def analyze_chart(indicated_data, df_analyzed):
    #quote_price = indicated_data.loc[:,'Adj Close'].iloc[-1]
    # Check RSI
    if indicated_data.loc[:,'rsi'].iloc[-1] < 35: 
        rsi = "Oversold"
    elif indicated_data.loc[:,'rsi'].iloc[-1] > 65: 
        rsi = "Overbought"
    else: 
        rsi = None

    # Check SMA Trend
    if indicated_data.loc[:,'sma30'].iloc[-1]<indicated_data.loc[:,'sma10'].iloc[-1]:
        trend = "Uptrend"
    elif indicated_data.loc[:,'sma30'].iloc[-1]>indicated_data.loc[:,'sma10'].iloc[-1]:
        trend = "Downtrend"
    else:
        trend = None
    
    # Check 200SMA
    if indicated_data.loc[:,'Open'].iloc[-1]>indicated_data.loc[:,'sma200'].iloc[-1]: 
        above200 = True
    else:
        above200 = None
    
    # Check for Earnings
    try:
        if pd.isnull(si.get_quote_table(ticker)['Earnings Date']):
            earnings_date = None
        elif datetime.strptime(si.get_quote_table(ticker)['Earnings Date'].split(' - ')[0], '%b %d, %Y'):
            earnings_date = datetime.strptime(si.get_quote_table(ticker)['Earnings Date'].split(' - ')[0], '%b %d, %Y')
        else:
            earnings_date = datetime.strptime(si.get_quote_table(ticker)['Earnings Date'], '%b %d, %Y')
    except:
        earnings_date = None
    
    # Check for support or resistance
    req = requests.get(f'https://finnhub.io/api/v1/scan/support-resistance?symbol={ticker}&resolution=D&token={finn_token}')
    supp_res = None
    supp_res_price = float()
    for level in req.json()['levels']:
        if float(level)*0.90 < indicated_data.loc[:,'Open'].iloc[-1] < float(level)*1.10:
            if indicated_data.loc[:,'Open'].iloc[-1] >= float(level):
                supp_res = "support"
                supp_res_price = round(level, 2)
            elif indicated_data.loc[:,'Open'].iloc[-1] <= float(level):
                supp_res = "resistance"
                supp_res_price = round(level, 2)
            else:
                supp_res = "Indeterminant"
                supp_res_price = None
        else:
            pass
    # Check TAZ
    # Check for Pullback
    if indicated_data.loc[:,'Adj Close'].iloc[-1]<= indicated_data.loc[:,'Adj Close'].iloc[-2]<= indicated_data.loc[:,'Adj Close'].iloc[-3]:
        pullback = True
    else: 
        pullback = None
    # Add latest sentiment
    

    df_analyzed = df_analyzed.append({'Ticker' : ticker, 
                          'Open' : round(indicated_data.loc[:,'Open'].iloc[-1]),
                          'Quote' : round(indicated_data.loc[:,'Adj Close'].iloc[-1]),
                          'RSI' : rsi,
                          'Trend' : trend,
                          'Above200' : above200,
                          'Earnings' : earnings_date, 
                          'Supp/Res' : supp_res,
                          'S/R Price' : supp_res_price,
                          'Pullback' : pullback,
                          'Sentiment' : sentiment
                         }, ignore_index=True)
    
    return df_analyzed

def analyze_position(df_analyzed, capital, risk):
    position_risk = capital*risk
    
    df_analyzed['Entry'] = df_analyzed['S/R Price']
    df_analyzed['Stoploss'] = df_analyzed['S/R Price'].astype(float).apply(lambda x: x * float(0.95))  
    df_analyzed['risk_per_share'] = df_analyzed['Entry'] - df_analyzed['Stoploss']
    df_analyzed['position_size'] = round(position_risk/df_analyzed['risk_per_share'])

    return df_analyzed


In [15]:
scanner_list = ['AAL', 'AES', 'AMCR', 'APA', 'BAC']
df_analyzed = pd.DataFrame(columns=['Ticker', 'Open', 'Quote', 'RSI', 'Trend', 'Above200', 'Earnings', 'Supp/Res', 'S/R Price', 'Pullback'])
for ticker in scanner_list:
    print(ticker)
    # Get historical data
    data = get_hist(ticker, conn)
    
    # Add indicator data
    indicated_data = get_indicators(data)
    
    # Analyze stonks:
    df_analyzed = analyze_chart(indicated_data, df_analyzed)
    
    df_analyzed = analyze_position(df_analyzed, capital, risk)

df_analyzed = df_analyzed[df_analyzed['Above200'] == True]
df_analyzed = df_analyzed[df_analyzed['RSI'] != None]
df_analyzed = df_analyzed[df_analyzed['Trend'] != None]
df_analyzed = df_analyzed[df_analyzed['Earnings'] != None]
df_analyzed = df_analyzed[df_analyzed['Supp/Res'] != None]
df_analyzed = df_analyzed[df_analyzed['Pullback'] != None]
print(df_analyzed)
#df_analyzed.to_excel('Output.xlsx', ignore_index=True)


AAL
AES
AMCR
APA
BAC
  Ticker Open Quote   RSI      Trend Above200   Earnings    Supp/Res  \
1    AES   19    19  None    Uptrend     True 2020-11-04  resistance   
2   AMCR   11    11  None  Downtrend     True 2020-08-18  resistance   

   S/R Price Pullback  Entry  Stoploss  risk_per_share  position_size  
1      19.12     None  19.12   18.1640          0.9560         5230.0  
2      11.59     None  11.59   11.0105          0.5795         8628.0  


                    Unnamed: 0 As of Date: 10/9/2020Current 6/30/2020  \
0      Market Cap (intraday) 5                       81.39B    46.42B   
1           Enterprise Value 3                       79.62B    45.14B   
2                 Trailing P/E                       282.31    159.00   
3                Forward P/E 1                       153.85    232.56   
4  PEG Ratio (5 yr expected) 1                         4.03      7.52   
5            Price/Sales (ttm)                        14.79      9.60   
6             Price/Book (mrq)                        42.25     25.68   
7   Enterprise Value/Revenue 3                        13.53     23.47   
8    Enterprise Value/EBITDA 6                        1.68k     1.91k   

  3/31/2020 12/31/2019 9/30/2019  
0    22.98B     27.08B    26.58B  
1    21.84B     26.31B    25.96B  
2     64.67        NaN       NaN  
3     72.46      67.11     55.25  
4      2.18       1.86      1.54  
5      5.18       6.08      6.54  
6     13.40      21.66  